In [4]:
!pip install -U sentence-transformers pandas

  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)


You should consider upgrading via the 'C:\Users\archi\Documents\UIC\421\paper-reproducibility\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import sys
print(sys.executable)

c:\Users\archi\Documents\UIC\421\paper-reproducibility\venv\Scripts\python.exe


In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer

c:\Users\archi\Documents\UIC\421\paper-reproducibility\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load your data seperated by semi color
df = pd.read_csv("HateWiC_IndividualAnnos.csv", sep=';')

In [8]:
# Load models
bert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
hatebert_model = SentenceTransformer('GroNLP/hateBERT')

c:\Users\archi\Documents\UIC\421\paper-reproducibility\venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\archi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
No sentence-transformers model found with name GroNLP/hateBERT. Creati

In [9]:
# Select a small sample
sample = df.sample(5, random_state=42)

In [10]:
# Generate embeddings
for idx, row in sample.iterrows():
    wic_text = row['example']
    def_text = row['wiktionary_definition']

    bert_wic = bert_model.encode(wic_text)
    bert_def = bert_model.encode(def_text)
    hatebert_wic = hatebert_model.encode(wic_text)
    hatebert_def = hatebert_model.encode(def_text)

    print(f"\n--- Sample {idx} ---")
    print(f"WiC Sentence: {wic_text}")
    print(f"Definition: {def_text}")
    print(f"BERT WiC Embedding Shape: {bert_wic.shape}")
    print(f"HateBERT WiC Embedding Shape: {hatebert_wic.shape}")


--- Sample 396 ---
WiC Sentence: ‘baniak baniak you ole fucker! | you’re tighter than hogan’s goat!’ | baniak only smiles | a perfect smile
Definition: A fool.
BERT WiC Embedding Shape: (384,)
HateBERT WiC Embedding Shape: (768,)

--- Sample 247 ---
WiC Sentence: The latter may become either Athenas or feminists. If one gets a certain sort of male sponsorship, becomes a Daddy's girl, one is allowed to function in these vocations of the righteous  […]
Definition: A woman who colludes with the patriarchy rather than actively opposing it.
BERT WiC Embedding Shape: (384,)
HateBERT WiC Embedding Shape: (768,)

--- Sample 2308 ---
WiC Sentence: We need to figure out how to balance the public interest in learning about a mass shooting with the public interest in reducing copycat crime.
Definition: Imitative; unoriginal.
BERT WiC Embedding Shape: (384,)
HateBERT WiC Embedding Shape: (768,)

--- Sample 1430 ---
WiC Sentence: Well, what do you think of the Canuck elections?
Definition: Canadian

In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [19]:
# Load your data seperated by semi color
hatewic_df = pd.read_csv("HateWiC_IndividualAnnos.csv", sep=';')

In [12]:
# Load FLAN-T5 for generating definitions
t5_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

c:\Users\archi\Documents\UIC\421\paper-reproducibility\venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\archi\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling 

In [14]:
def get_wic_and_def_embeddings(row, model):
    wic_emb = model.encode(row["example"], convert_to_numpy=True)
    def_emb = model.encode(row["wiktionary_definition"], convert_to_numpy=True)
    return wic_emb, def_emb


In [13]:
import torch

def generate_t5_definition(sentence, term):
    prompt = f"{sentence} What is the definition of {term}?"
    input_ids = t5_tokenizer(prompt, return_tensors="pt").input_ids
    outputs = t5_model.generate(input_ids, max_new_tokens=50)
    return t5_tokenizer.decode(outputs[0], skip_special_tokens=True)


In [15]:
def get_t5def_embedding(def_text, model):
    return model.encode(def_text, convert_to_numpy=True)


In [16]:
def get_ann_embedding(row, model):
    return model.encode(row["annotator_profile_description"], convert_to_numpy=True)


In [17]:
def build_embeddings(df, model, use_t5=False, include_ann=False):
    results = []
    for idx, row in df.iterrows():
        try:
            wic_emb, def_emb = get_wic_and_def_embeddings(row, model)

            t5def_emb = np.zeros_like(def_emb)
            if use_t5:
                t5_text = generate_t5_definition(row["example"], row["term"])
                t5def_emb = get_t5def_embedding(t5_text, model)

            ann_emb = np.zeros_like(def_emb)
            if include_ann:
                ann_emb = get_ann_embedding(row, model)

            # Concatenate based on settings
            combined = np.concatenate([wic_emb, def_emb, t5def_emb, ann_emb])
            results.append((combined, row["binary_label"]))
        except Exception as e:
            print(f"Error at row {idx}: {e}")
    return results


In [22]:
import numpy as np

sample_df = hatewic_df.sample(5, random_state=0)
results = build_embeddings(sample_df, model=bert_model, use_t5=True, include_ann=True)

for vec, label in results:
    print(f"Embedding shape: {vec.shape}, Label: {label}")


Embedding shape: (1536,), Label: 0.0
Embedding shape: (1536,), Label: 1.0
Embedding shape: (1536,), Label: 0.0
Embedding shape: (1536,), Label: 0.0
Embedding shape: (1536,), Label: 1.0


In [25]:
# For majority task (BERT):
bert_vectors = build_embeddings(hatewic_df, model=bert_model, use_t5=True, include_ann=False)

# For subjective task (HateBERT):
hatebert_vectors = build_embeddings(hatewic_df, model=hatebert_model, use_t5=True, include_ann=True)


KeyboardInterrupt: 

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import numpy as np

def evaluate_mlp(X, y, hidden_layer_sizes=(300, 200, 100, 50), max_iter=10, lr=0.0005, folds=10):
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    accuracies = []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        clf = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                            max_iter=max_iter,
                            learning_rate_init=lr,
                            early_stopping=True,
                            random_state=42)

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)
        print(f"Fold {fold + 1} Accuracy: {acc:.4f}")

    print(f"\n✅ Mean Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
    return accuracies


In [24]:
# Assume your data is stored as:
# results = [(embedding_vec, label), ...]

X = np.array([r[0] for r in results])
y = np.array([int(float(r[1])) for r in results])  # ensure labels are int

# Run evaluation
evaluate_mlp(X, y)


ValueError: Cannot have number of splits n_splits=10 greater than the number of samples: n_samples=5.